In [1]:
### 基础导包
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [10]:
### 模型导包
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score 

In [3]:
### 基础设置
pd.set_option('display.float_format',lambda x: '%.2f'%x)
pd.set_option('display.expand_frame_repr',False)
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
#pd.set_option('display.max_rows', None)
pd.set_eng_float_format(accuracy=1, use_eng_prefix=True)

In [4]:
train = pd.read_csv("./processed/train_for_model.csv")
test = pd.read_csv("./processed/test_for_model.csv")


In [5]:
def split_data(data,y_feature):
    return train_test_split(data,data[y_feature],test_size=0.20,random_state=20220319)
    raise NotImplementedError

In [6]:
### 注意只对train划出验证集来，test按兵不动
x_train,x_eval,y_train,y_eval = split_data(train,'Survived')   # 注意，train 里面含有标签 ，因此划分完毕之后应该手动去除标签
x_train.drop(columns=['Survived'] , axis= 0 , inplace=True) 
x_eval.drop(columns=['Survived'] , axis= 0 , inplace=True) 

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
params = { 
'learning_rate' : 0.02 ,
'depth' : 13 ,
'bootstrap_type' : 'Bernoulli' ,
'od_type' : 'Iter' ,
'od_wait' : 50 ,
'random_state' : 20220319 
}


In [15]:
cat_model = CatBoostClassifier( 
        iterations=20000 ,
        eval_metric = 'Logloss' ,
        **params
)

In [16]:
cat_model.fit(x_train , 
                y_train ,
                eval_set = (x_eval , y_eval) , 
                cat_features = [] ,
                use_best_model = True ,
                verbose = False )

In [18]:
y_pre = cat_model.predict(test) 

In [19]:
submit = pd.read_csv("./work/gender_submission.csv")
submit["Survived"] =  y_pre

In [20]:
submit.to_csv("./submit/submit_cat.csv" , index=0)  # 将预测文件保存